![rmotr](https://i.imgur.com/jiPp4hj.png)
<hr style="margin-bottom: 40px;">

<img src="https://user-images.githubusercontent.com/7065401/39117440-24199c72-46e7-11e8-8ffc-25c6e27e07d4.jpg"
    style="width:300px; float: right; margin: 0 40px 40px 40px;"></img>

# Handling Missing Data with Pandas

pandas borrows all the capabilities from numpy selection + adds a number of convenient methods to handle missing values. Let's see one at a time:

![separator2](https://i.imgur.com/4gX5WFr.png)

## Hands on!

In [49]:
import numpy as np
import pandas as pd

### Pandas utility functions

Similarly to `numpy`, pandas also has a few utility functions to identify and detect null values:

In [50]:
pd.isnull(np.nan)

True

In [51]:
pd.isnull(None)

True

In [52]:
pd.isna(np.nan)

True

In [53]:
pd.isna(None)

True

The opposite ones also exist:

In [54]:
pd.notnull(None)

False

In [55]:
pd.notnull(np.nan)

False

In [56]:
pd.notna(None)

False

In [57]:
pd.notnull(3)

True

These functions also work with Series and `DataFrame`s:

In [58]:
a = pd.Series([1, 2, 3, np.nan, None, 4], dtype='float')

In [59]:
pd.isnull(a)

,0
0,False
1,False
2,False
3,True
4,True
5,False


In [60]:
pd.notnull(a)

,0
0,True
1,True
2,True
3,False
4,False
5,True


In [61]:
b = pd.DataFrame({
    'Column A': [1, np.nan, 7],
    'Column B': [np.nan, 2, 3],
    'Column C': [np.nan, 2, np.nan]
})

In [62]:
pd.isnull(b)

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


![separator1](https://i.imgur.com/ZUWYTii.png)

### Pandas Operations with Missing Values

Pandas manages missing values more gracefully than numpy. `nan`s will no longer behave as "viruses", and operations will just ignore them completely:

In [63]:
a.count()

np.int64(4)

In [64]:
a.sum()

np.float64(10.0)

In [65]:
a.mean()

np.float64(2.5)

### Filtering missing data

As we saw with numpy, we could combine boolean selection + `pd.isnull` to filter out those `nan`s and null values:

In [66]:
pd.notnull(b)

,Column A,Column B,Column C
0,True,False,False
1,False,True,True
2,True,True,False


In [67]:
pd.isnull(b)

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


In [68]:
pd.notnull(b).sum()

,0
Column A,2
Column B,2
Column C,1


In [69]:
pd.isnull(b).sum()

,0
Column A,1
Column B,1
Column C,2


In [70]:
b[pd.notnull(b)]

,Column A,Column B,Column C
0,1.0,NaN,NaN
1,NaN,2.0,2.0
2,7.0,3.0,NaN


But both `notnull` and `isnull` are also methods of `Series` and `DataFrame`s, so we could use it that way:

In [71]:
a.isnull()

,0
0,False
1,False
2,False
3,True
4,True
5,False


In [72]:
b.notnull()

,Column A,Column B,Column C
0,True,False,False
1,False,True,True
2,True,True,False


In [73]:
b[b.notnull()]

,Column A,Column B,Column C
0,1.0,NaN,NaN
1,NaN,2.0,2.0
2,7.0,3.0,NaN


![separator1](https://i.imgur.com/ZUWYTii.png)

### Dropping null values

Boolean selection + `notnull()` seems a little bit verbose and repetitive. And as we said before: any repetitive task will probably have a better, more DRY way. In this case, we can use the `dropna` method:

In [74]:
a

,0
0,1.0
1,2.0
2,3.0
3,NaN
4,NaN
5,4.0


In [75]:
a.dropna()

,0
0,1.0
1,2.0
2,3.0
5,4.0


### Dropping null values on DataFrames

You saw how simple it is to drop `na`s with a Series. But with `DataFrame`s, there will be a few more things to consider, because you can't drop single values. You can only drop entire columns or rows. Let's start with a sample `DataFrame`:

In [76]:
df = pd.DataFrame({
    'Column A': [1, np.nan, 30, np.nan],
    'Column B': [2, 8, 31, np.nan],
    'Column C': [np.nan, 9, 32, 100],
    'Column D': [5, 8, 34, 110],
})

In [77]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [78]:
df.shape

(4, 4)

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Column A  2 non-null      float64
 1   Column B  3 non-null      float64
 2   Column C  3 non-null      float64
 3   Column D  4 non-null      int64  
dtypes: float64(3), int64(1)
memory usage: 260.0 bytes


In [80]:
df.isnull()

,Column A,Column B,Column C,Column D
0,False,False,True,False
1,True,False,False,False
2,False,False,False,False
3,True,True,False,False


In [81]:
df.isnull().sum()

,0
Column A,2
Column B,1
Column C,1
Column D,0


The default `dropna` behavior will drop all the rows in which _any_ null value is present:

In [82]:
df.dropna()

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


In this case we're dropping **rows**. Rows containing null values are dropped from the DF. You can also use the `axis` parameter to drop columns containing null values:

In [83]:
df.dropna(axis=1)  # axis='columns' also works

,Column D
0,5
1,8
2,34
3,110


In this case, any row or column that contains **at least** one null value will be dropped. Which can be, depending on the case, too extreme. You can control this behavior with the `how` parameter. Can be either `'any'` or `'all'`:

In [84]:
df2 = pd.DataFrame({
    'Column A': [1, np.nan, 30],
    'Column B': [2, np.nan, 31],
    'Column C': [np.nan, np.nan, 100]
})

In [85]:
df2

,Column A,Column B,Column C
0,1.0,2.0,NaN
1,NaN,NaN,NaN
2,30.0,31.0,100.0


In [86]:
df.dropna(how='all')

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [87]:
df.dropna(how='any')  # default behavior

,Column A,Column B,Column C,Column D
2,30.0,31.0,32.0,34


You can also use the `thresh` parameter to indicate a _threshold_ (a minimum number) of non-null values for the row/column to be kept:

In [88]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [89]:
df.dropna(thresh=3)

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34


In [90]:
df.dropna(thresh=3, axis='columns')

,Column B,Column C,Column D
0,2.0,NaN,5
1,8.0,9.0,8
2,31.0,32.0,34
3,NaN,100.0,110


![separator1](https://i.imgur.com/ZUWYTii.png)

### Filling null values

Sometimes instead than dropping the null values, we might need to replace them with some other value. This highly depends on your context and the dataset you're currently working. Sometimes a `nan` can be replaced with a `0`, sometimes it can be replaced with the `mean` of the sample, and some other times you can take the closest value. Again, it depends on the context. We'll show you the different methods and mechanisms and you can then apply them to your own problem.

In [91]:
s

NameError: name 's' is not defined

**Filling nulls with a arbitrary value**

In [93]:
b.fillna(0)

,Column A,Column B,Column C
0,1.0,0.0,0.0
1,0.0,2.0,2.0
2,7.0,3.0,0.0


In [95]:
b.fillna(b.mean())

,Column A,Column B,Column C
0,1.0,2.5,2.0
1,4.0,2.0,2.0
2,7.0,3.0,2.0


In [96]:
b

,Column A,Column B,Column C
0,1.0,NaN,NaN
1,NaN,2.0,2.0
2,7.0,3.0,NaN


**Filling nulls with contiguous (close) values**

The `method` argument is used to fill null values with other values close to that null one:

In [100]:
b

,Column A,Column B,Column C
0,1.0,NaN,NaN
1,NaN,2.0,2.0
2,7.0,3.0,NaN


In [97]:
b.fillna(method='ffill')
# b.ffill() better

/tmp/ipython-input-97-1779961045.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  b.fillna(method='ffill')


,Column A,Column B,Column C
0,1.0,NaN,NaN
1,1.0,2.0,2.0
2,7.0,3.0,2.0


In [99]:
b.bfill()
# b.fillna(method='bfill')

,Column A,Column B,Column C
0,1.0,2.0,2.0
1,7.0,2.0,2.0
2,7.0,3.0,NaN


This can still leave null values at the extremes of the Series/DataFrame:

In [101]:
pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')

/tmp/ipython-input-101-533636215.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pd.Series([np.nan, 3, np.nan, 9]).fillna(method='ffill')


,0
0,NaN
1,3.0
2,3.0
3,9.0


In [102]:
pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method='bfill')

/tmp/ipython-input-102-2629518188.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  pd.Series([1, np.nan, 3, np.nan, np.nan]).fillna(method='bfill')


,0
0,1.0
1,3.0
2,3.0
3,NaN
4,NaN


### Filling null values on DataFrames

The `fillna` method also works on `DataFrame`s, and it works similarly. The main differences are that you can specify the `axis` (as usual, rows or columns) to use to fill the values (specially for methods) and that you have more control on the values passed:

In [103]:
df

,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,NaN,8.0,9.0,8
2,30.0,31.0,32.0,34
3,NaN,NaN,100.0,110


In [104]:
df.fillna({'Column A': 0, 'Column B': 99, 'Column C': df['Column C'].mean()})

,Column A,Column B,Column C,Column D
0,1.0,2.0,47.0,5
1,0.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,0.0,99.0,100.0,110


In [105]:
df.fillna(method='ffill', axis=0)

/tmp/ipython-input-105-3122255136.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=0)


,Column A,Column B,Column C,Column D
0,1.0,2.0,NaN,5
1,1.0,8.0,9.0,8
2,30.0,31.0,32.0,34
3,30.0,31.0,100.0,110


In [106]:
df.fillna(method='ffill', axis=1)

/tmp/ipython-input-106-901808322.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=1)


,Column A,Column B,Column C,Column D
0,1.0,2.0,2.0,5.0
1,NaN,8.0,9.0,8.0
2,30.0,31.0,32.0,34.0
3,NaN,NaN,100.0,110.0


![separator1](https://i.imgur.com/ZUWYTii.png)

### Checking if there are NAs

The question is: Does this `Series` or `DataFrame` contain any missing value? The answer should be yes or no: `True` or `False`. How can you verify it?

**Example 1: Checking the length**

If there are missing values, `s.dropna()` will have less elements than `s`:

In [109]:
b.dropna().count()

,0
Column A,0
Column B,0
Column C,0


In [112]:
b.count()

,0
Column A,2
Column B,2
Column C,1


In [111]:
missing_values = len(b.dropna()) != len(b)
missing_values

True

There's also a `count` method, that excludes `nan`s from its result:

In [113]:
len(b)

3

In [114]:
b.count()

,0
Column A,2
Column B,2
Column C,1


So we could just do:

In [115]:
missing_values = b.count() != len(b)
missing_values

,0
Column A,True
Column B,True
Column C,True


**More Pythonic solution `any`**

The methods `any` and `all` check if either there's `any` True value in a Series or `all` the values are `True`. They work in the same way as in Python:

In [116]:
pd.Series([True, False, False]).any()

np.True_

In [120]:
pd.Series([True, False, False]).all()

np.False_

In [118]:
pd.Series([True, True, True]).all()

np.True_

The `isnull()` method returned a Boolean `Series` with `True` values wherever there was a `nan`:

In [121]:
b.isnull()

,Column A,Column B,Column C
0,False,True,True
1,True,False,False
2,False,False,True


So we can just use the `any` method with the boolean array returned:

In [122]:
pd.Series([1, np.nan]).isnull().any()

np.True_

In [123]:
pd.Series([1, 2]).isnull().any()

np.False_

In [124]:
b.isnull().any()

,0
Column A,True
Column B,True
Column C,True


A more strict version would check only the `values` of the Series:

In [126]:
b.isnull().values

array([[False,  True,  True],
       [ True, False, False],
       [False, False,  True]])

In [127]:
b.isnull().values.any()

np.True_

![separator2](https://i.imgur.com/4gX5WFr.png)